In [29]:
from dotenv import load_dotenv
load_dotenv()
import os
import google.generativeai as genai
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

In [ ]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=768, metric="cosine", spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  ),
  deletion_protection="disabled"
)

In [31]:
import json 
data = json.load(open("reviews.json"))

data["reviews"]



[{'state': 'California',
  'city': 'Los Angeles',
  'stars': 5,
  'review': 'A vibrant city with endless entertainment and fantastic weather!'},
 {'state': 'New York',
  'city': 'New York City',
  'stars': 4,
  'review': 'Great for nightlife and culture, but traffic is a nightmare.'},
 {'state': 'Texas',
  'city': 'Austin',
  'stars': 5,
  'review': 'A fun, music-filled city with amazing food and welcoming people.'},
 {'state': 'Illinois',
  'city': 'Chicago',
  'stars': 4,
  'review': 'Beautiful architecture and deep-dish pizza, but winters are harsh.'},
 {'state': 'Florida',
  'city': 'Miami',
  'stars': 3,
  'review': "Lovely beaches, but it's too hot and humid most of the year."},
 {'state': 'Nevada',
  'city': 'Las Vegas',
  'stars': 4,
  'review': 'Perfect for a quick getaway, though it can get overwhelming.'},
 {'state': 'Colorado',
  'city': 'Denver',
  'stars': 5,
  'review': 'Great for outdoor enthusiasts, with stunning views and friendly locals.'},
 {'state': 'Washington',
 

In [ ]:
#create embeddings

genai.configure(api_key=os.getenv("GEMINIAI_API_KEY"))
processed_data = []
client = genai

# Assuming 'client' is already instantiated and correctly configured
for review in data['reviews']:
    # Ensure the review text is being passed correctly
    model = "models/text-embedding-004"
    response = genai.embed_content(
                                model=model,
                                content=review['review'],
                                #task_type="retrieval_document",
                                )
    # response = client.generate_embeddings(
    #     input=review['review'],
    #     model="text-embedding-005",

    # )

    print(response)
    embedding = response.data[0].embedding # Assuming the response contains the embeddings
    
    # Process or store the embeddings as needed


    #individual piece of data
    processed_data.append({
        "values": embedding,
        "id": review["state"],
        "metadata": {
            "review": review["review"],
            "city": review["city"],
            "stars" : review["stars"]
        }
    })


{'embedding': [-0.0015006175, 0.011338981, 0.038019978, 0.0050780396, 0.0067466646, -0.052020676, 0.00079473, -0.02144273, 0.028515698, 0.05014162, -0.05492301, 0.05010295, 0.02082227, -0.010123367, 0.03807996, -0.022801172, -0.0471117, 0.014552404, -0.15191309, -0.038748544, 0.023211718, -0.0009474588, 0.0730699, -0.051709313, 0.023414392, -0.021215925, 0.0123841725, 0.017970411, 0.033031214, 0.036283955, -0.016841326, 0.04066232, 0.0056357277, -0.013797509, -0.03638691, 0.004581084, 0.027998919, -0.07366585, 0.022236187, 0.0016244107, -0.085761204, 0.03354654, -0.02503103, -0.024930825, -0.04572523, -0.07046531, -0.023535566, 0.022041177, 0.013015534, 0.042669058, 0.016743833, 0.012892009, 0.004989652, 0.03410962, -0.018607266, -0.01902047, -0.05642336, 0.0149620995, 0.11964834, 0.013683578, 0.027470585, 0.005106622, 0.01632254, -0.019074716, 0.043895733, -0.054455888, -0.04429278, 0.04865147, 0.02387023, -0.009263117, -0.013807383, 0.04779293, -0.008933326, 0.044666134, 0.03330619, 

AttributeError: 'dict' object has no attribute 'data'

In [46]:
processed_data[0]

{'values': {'embedding': [0.012304764,
   0.018752133,
   0.008979598,
   -0.03754818,
   0.018326351,
   -0.04676483,
   -0.006248248,
   -0.0256721,
   0.020411972,
   0.049672578,
   -0.028578928,
   0.024019217,
   0.0634724,
   0.0021423546,
   0.013359313,
   -0.0055283788,
   -0.01875836,
   0.006114825,
   -0.12044967,
   -0.013200333,
   0.023251988,
   -0.029555164,
   0.055228706,
   -0.01968303,
   -0.006913252,
   -0.0056698527,
   -0.006085882,
   0.022584595,
   -0.0034563069,
   0.024124064,
   0.01231061,
   0.06433503,
   0.0056542116,
   -0.034544364,
   -0.019815378,
   0.0023503513,
   0.027825987,
   -0.08198869,
   0.045915235,
   -0.02187665,
   -0.06927587,
   0.06391765,
   0.0020550808,
   0.0075251674,
   -0.0600888,
   -0.05789757,
   -0.0265676,
   0.025426881,
   -0.02059692,
   0.04785431,
   -0.009365851,
   0.038206343,
   -0.025013816,
   0.06525866,
   -0.042699542,
   -0.007207529,
   -0.050299663,
   0.0076239053,
   0.069754355,
   0.011871056,
  

In [52]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"

)

TypeError: Column `values` is expected to be a list of floats